In [2]:
import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()
# Access the API key from the environment
api_key = os.getenv("REPLICATE_API_TOKEN")


In [4]:
# We use this variable to control where you want to host LLaVA, locally or remotely?
# More details in the two setup options below.
import json
import os
import random
import time
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import matplotlib.pyplot as plt
import requests
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.llava_agent import LLaVAAgent, llava_call

LLAVA_MODE = "remote"  # Either "local" or "remote"
assert LLAVA_MODE in ["local", "remote"]

if LLAVA_MODE == "remote":
    import replicate

    llava_config_list = [
        {
            "model": "yorickvp/llava-13b ",  # The model name doesn't matter here right now.
            "api_key": api_key,  # Note that you have to setup the API key with os.environ["REPLICATE_API_TOKEN"]
            "base_url": "yorickvp/llava-13b:2facb4a474a0462c15041b78b1ad70952ea46b5ec6ad29583c0b29dbd4249591",
        }
    ]


# rst = llava_call(
#     "Describe this AutoGen framework <img https://th.bing.com/th/id/R.422068ce8af4e15b0634fe2540adea7a?rik=y4OcXBE%2fqutDOw&pid=ImgRaw&r=0.png> with bullet points.",
#     llm_config={"config_list": llava_config_list, "temperature": 0},
# )

# print(rst)
rst = llava_call(
    """based on this Categories [
    Categories
    1. Background Image
    2. Logo
    3. Call-To-Action (CTA) Button
    4. Icon
    5. Product Image
    6. Text Elements
    7. Infographic
    8. Banner
    9. Illustration
    10. Photograph
    11. Mascot
    12. Testimonial Quotes
    13. Social Proof
    14. Seal or Badge
    15. Graphs and Charts
    16. Decorative Elements
    17. Interactive Elements
    18. Animation Frames
    19. Coupon or Offer Code
    20. Legal Disclaimers or Terms
    21. Contact Information
    22. Map or Location Image
    23. QR Code]classify this <img ../image for test/content3.png>.""",
    llm_config={"config_list": llava_config_list, "temperature": 0},
)

print(rst)



Error: ReplicateError Details:
title: Free time limit reached
status: 402
detail: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.
None


In [5]:
import os
import random
from pathlib import Path

# Define the agents
image_agent = LLaVAAgent(
    name="image-creator",
    max_consecutive_auto_reply=10,
    llm_config={"config_list": llava_config_list, "temperature": 0, "max_new_tokens": 1000},
)

user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
    human_input_mode="NEVER",
    max_consecutive_auto_reply=0,
)

# Set the input and output directories
input_dir = "../src/image"
output_dir = "output_test"
os.makedirs(output_dir, exist_ok=True)

# Collect image paths
image_paths = list(Path(input_dir).glob("*"))
if len(image_paths) < 3:
    raise ValueError("Not enough images in the input directory to create a storyboard. At least 3 images are required.")

# Randomly select three images
selected_images = random.sample(image_paths, 3)

# Create a message including the paths of the selected images
message1 = "Using the following 3 images, make a storyboard of 3 frames by writing a code :\n"
for image_path in selected_images:
    message =message1 + f"<img {image_path}>\n"

# Send the images to the image_agent for classification
try:
    print(f"Sending message to image agent:\n{message}")
    
    response = user_proxy.initiate_chat(
        image_agent,
        message=message
    )

    print(f"Response from image agent: {response}")
    
except Exception as e:  

    print(f"An error occurred: {e}")


Sending message to image agent:
Using the following 3 images, make a storyboard of 3 frames by writing a code :
<img ../src/image/cheese.png>

User_proxy (to image-creator):

Using the following 3 images, make a storyboard of 3 frames by writing a code :
<image>


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
You are an AI agent and you can view images.
###Human: Using the following 3 images, make a storyboard of 3 frames by writing a code :
<image>


###Assistant: 
Error: ReplicateError Details:
title: Free time limit reached
status: 402
detail: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.
An error occurred: object of type 'NoneType' has no len()
